In [118]:
"""
Correct cGENIE field of sed_CaCO3_13C

Some grids have illness d13C data of -44, -311.3, 9.7, 15.1
at locations of 35E,45E,55E and 59.4N,66.4N,76.5N
"""
import yaml
import numpy as np
import pandas
import os
from netCDF4 import Dataset
from shutil import copyfile

nc_field = 'sed_CaCO3_13C'
nc_field_ref = 'sed_CaCO3'

config_dir = '/mnt/c/Users/mul450/Dropbox/git/deepDA/'
config_name = "DeepDA_config.yml"
#config_name = "petmproxy3slices_v0.0.10gt1.csvexp_petm78_og1_qc_obs_20200203_test2.yml"
f = open(config_dir+config_name, 'r')
yml_dict = yaml.load(f, Loader=yaml.FullLoader)
f.close()

In [119]:
def check_ill(x):
    x_ill = np.asarray(np.where( np.logical_or(x<0, np.logical_and(x>10, x<1.0e10))))
    #print(x_ill)
    if x_ill.shape[1] > 0:
        return True
    else:
        return False
def irrigate_ill(x):
    x_ill = np.asarray(np.where( np.logical_or(x<0, np.logical_and(x>10, x<1.0e10))))
    x_ill_len = x_ill.shape[1]
    for j in range(x_ill_len):
        ind0 = x_ill[0][j]
        ind1 = x_ill[1][j]
        x_s = np.array([[ind0-1,ind0,ind0+1,ind0-1,ind0+1,ind0-1,ind0,ind0+1], [ind1-1,ind1-1,ind1-1,ind1,ind1,ind1+1,ind1+1,ind1+1]])
        x_s[x_s == 36] = -1
        x_re = np.full(x_s.shape[1],np.nan)
        for k in range(x_s.shape[1]):
            if ~np.ma.is_masked(x[x_s[0][k], x_s[1][k]]):
                if x[x_s[0][k], x_s[1][k]] < 10 and x[x_s[0][k], x_s[1][k]] > 0:
                    x_re[k] = x[x_s[0][k], x_s[1][k]]
        x[ind0][ind1] = np.nanmean(x_re)
    return x

In [120]:
dir_prior = yml_dict['core']['prior_dir']
dir_prior_full = os.listdir(dir_prior)
print(dir_prior_full[0])
prior_len = len(dir_prior_full)
#prior_len = 1
# ========= dataset for plot =========
cGENIEGrid = yml_dict['core']['proj_dir'] + '/data_misc/cGENIEGrid.csv'
cGENIEGrid = pandas.read_csv(cGENIEGrid)
cGENIEGridB_lat36 = cGENIEGrid['lat']
cGENIEGridB_lon36 = cGENIEGrid['lon']

for i in range(prior_len):
    name_nc_2d = dir_prior+'/'+ dir_prior_full[i]+'/'+ 'biogem' + '/' + 'fields_biogem_2d.nc'
    x = Dataset(name_nc_2d).variables[nc_field][0,:,:]
    x_ill = check_ill(x)
    if x_ill:
        x_ill_index = np.asarray(np.where(x < -9.0e+18))
        print('>> {} Data is ill. try #1 to fix illness index '.format(dir_prior_full[i]))
        #print('{}'.format(x_ill_index))
        x = np.copy(irrigate_ill(x))
    x_ill = check_ill(x)
    if x_ill:
        x_ill_index = np.asarray(np.where(x < -9.0e+18))
        print('x x   Data is ill. still need to fix illness index ######################################################'.format())
        print('{}'.format(x_ill_index))
    else:
        namc_nc_cp = dir_prior+'/'+ dir_prior_full[i]+'/'+ 'biogem' + '/' + 'fields_biogem_2d_13ccorr.nc'
        print('√ √   New 2d file: {}'.format(namc_nc_cp))
        copyfile(name_nc_2d, namc_nc_cp)
        nf = Dataset(namc_nc_cp, 'a', format='NETCDF4')
        sed_CaCO3_13C_corr = nf.createVariable('sed_CaCO3_13C_corr', 'f4', ('time','lat', 'lon'))
        sed_CaCO3_13C_corr[:,:,:] = np.copy(x.reshape(1,36,36))
        sed_CaCO3_13C_corr.units = 'o/oo'
        nf.close()  # close the new file
print('>>  End writing netCDF')

ML.petm009.ID.1
>> ML.petm009.ID.1 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.1/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.10 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.10/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.100 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.100/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.101 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.101/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.102 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.102/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.103 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.103/biogem/fields_bio

/home/mul450/miniconda3/envs/lmr_py3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice
/home/mul450/miniconda3/envs/lmr_py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  
/home/mul450/miniconda3/envs/lmr_py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


>> ML.petm009.ID.105 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.105/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.106 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.106/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.107 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.107/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.108 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.108/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.109 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.109/biogem/fields_biogem_2d_13ccorr.nc
>> ML.petm009.ID.11 Data is ill. try #1 to fix illness index 
√ √   New 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.11/biogem/fields_biogem_2d_13cco

In [100]:
nf.close()  # close the new file